<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Inheritance

In [1]:
# reload external dependencies when they change
%load_ext autoreload
%autoreload 2

%matplotlib inline

Suppose we define a `BankAccount` class as in the exercise in the [Classes and Objects](../modules/classes_and_objects.ipunb) module. Then we remember that the bank offers checking accounts and savings accounts that slightly have different rules. For instance, a customer can only make six withdrawals in one calendar month from a savings account. Let's suppose also that the fee for an overdraft is \\$10 for a savings account and \\$30 for a checking account.

One option for dealing with this situation would be to make two entirely separate `CheckingAccount` and `SavingsAccount` classes. However, on that approach we would have repeated code in the two classes. Writing that same code twice would not be hard, but accumulating repeated code will eventually kill a codebase by making it nearly impossible to reason about and to modify.

Thankfully, we have another option. We can make a `BankAccount` **parent class** that represents the elements that are common to checking accounts and savings accounts, and then create `CheckingAccount` and `SavingsAccount` **child classes** that **inherit from** `BankAccount`.

```python
class BankAccount:
    def __init__(self, owner_name, balance, fees_owed, overdraft_fee):
        self.owner_name = owner_name
        self.balance = balance
        self.fees_owed = fees_owed
        self.overdraft_fee = overdraft_fee

    def deposit(self, amount):
        if amount <= 0:
            raise ValueError("Deposit amount must be positive")
        else:
            self.balance += amount

    def withdraw(self, amount):
        if amount <= 0:
            raise ValueError("Withdrawal amount must be positive")
        else:
            self.balance -= amount
        if self.balance < 0:
            self.fees_owed += self.overdraft_fee

    def pay_fees(self, amount):
        if amount <= 0:
            raise ValueError("Withdrawal amount must be positive")
        elif amount > self.fees_owed:
            raise ValueError(
                f"Fee payment cannot exceed `fees_owed` amount of {self.fees_owed}"
            )
        else:
            self.fees_owed -= amount
```

We will define the `CheckingAccount` class to inherit from the `BankAccount` class, meaning that it gets all of `BankAccount`'s methods and attributes except where we overwrite them.

```python
class CheckingAccount(BankAccount):
    def __init__(self, owner_name, balance, fees_owed):
        super().__init__(owner_name, balance, fees_owed, overdraft_fee=30)
```

**Note:** `super()` refers to the class that we are inheriting from. For instance, here we want to initialize `CheckingAccount`s in the same way as `BankAccount`s but with a specific overdraft fee, so inside `CheckingAccount.__init__` we call `super().__init__` (meaning `BankAccount.__init__`), passing along the `owner_name`, `balance`, `fees_owed` values that we received as well as the hard-coded value of 30 for `overdraft_fee`.

*Import our classes*

In [2]:
from bank_account_with_inheritance import BankAccount, CheckingAccount

In [3]:
# Run this cell to test our code
import pytest


checking_account = CheckingAccount(owner_name="Finley Smith", balance=2000, fees_owed=0)

with pytest.raises(ValueError):
    checking_account.deposit(-10)

with pytest.raises(ValueError):
    checking_account.withdraw(-10)

with pytest.raises(ValueError):
    checking_account.pay_fees(-10)

assert checking_account.owner_name == "Finley Smith"
assert checking_account.balance == 2_000
assert checking_account.fees_owed == 0

checking_account.deposit(500)
assert checking_account.balance == 2_500

checking_account.withdraw(1_000)
assert checking_account.balance == 1_500

checking_account.withdraw(2_000)
assert checking_account.balance == -500

assert checking_account.fees_owed == 30

checking_account.pay_fees(15)
assert checking_account.fees_owed == 15

checking_account.pay_fees(15)
assert checking_account.fees_owed == 0

with pytest.raises(ValueError):
    checking_account.pay_fees(10)

*Add the `SavingsAccount` class*

```python
from datetime import datetime


class NotPermittedError(Exception):
    pass


class SavingsAccount(BankAccount):
    def __init__(self, owner_name, balance, fees_owed):
        super().__init__(owner_name, balance, fees_owed, overdraft_fee=10)
        self._last_withdrawal_month = datetime.today().month
        self._withdrawals_in_last_withdrawal_month = 0

    def withdraw(self, amount):
        if self._last_withdrawal_month == datetime.today().month:
            self._withdrawals_in_last_withdrawal_month += 1
            if self._withdrawals_in_last_withdrawal_month >= 6:
                raise NotPermittedError(
                    "Customer has used all allowed transactions for the month"
                )
        else:
            self._last_withdrawal_month = datetime.today().month
            self._withdrawals_in_last_withdrawal_month = 1

        super().withdraw(amount)
```

**Notes:**

- Using an underscore at the start of an attribute name in Python understood to be a signal that the attribute is intended to be used only inside the class definition. We can do the same thing with methods and with functions and classes that are intended to be used only within a module.
- We can define a custom error type by creating a class that inherits from `Exception`.

**Exercise**

*Time:* 5 mins.\
*Format:* Pairs\
*Post answers:* Yes

- Write tests for `SavingsAccount` like the tests above for `CheckingAccount` but with appropriate changes to account for the differences between the classes.

*Import our classes*

In [4]:
from bank_account_with_inheritance import SavingsAccount

In [5]:
savings_account = SavingsAccount(owner_name="Finley Smith", balance=2000, fees_owed=0)

In [6]:
with pytest.raises(ValueError):
    savings_account.deposit(-10)

with pytest.raises(ValueError):
    savings_account.withdraw(-10)

with pytest.raises(ValueError):
    savings_account.pay_fees(-10)

assert savings_account.owner_name == "Finley Smith"
assert savings_account.balance == 2_000
assert savings_account.fees_owed == 0

savings_account.deposit(500)
assert savings_account.balance == 2_500

savings_account.withdraw(1_000)
assert savings_account.balance == 1_500

savings_account.withdraw(2_000)
assert savings_account.balance == -500

assert savings_account.fees_owed == 10

savings_account.pay_fees(5)
assert savings_account.fees_owed == 5

savings_account.pay_fees(5)
assert savings_account.fees_owed == 0

with pytest.raises(ValueError):
    savings_account.pay_fees(10)

$\blacksquare$

**Note:** Having one method with different implementations for different child classes (such as `withdraw` in this case) is called **polymorphism**.

## Abstract Parent Classes

You can use a parent class to implement default behavior, but **sometimes there is no reasonable default behavior**. For instance, suppose you want to be able to create `Rectangle` and `Triangle` objects and calculate their areas. You could make a `Shape` base class, but what would you put inside it?

A good solution to this problem is to go ahead and create a `Shape` parent class, but to raise `NotImplementedError`s for each method that doesn't have a sensible default.

```python
class Shape:
    def __init__(self):
        raise NotImplementedError

    def area(self):
        raise NotImplementedError


class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def area(self):
        return self.width * self.height
```

`Shape` is not useful by itself: it cannot even be instantiated! However, it still helps us by providing a template for its child classes: we can see that each child class needs to implement `__init__` and `area`, and if it doesn't then we will get appropriate errors.

**Note:** A class like `Shape` that is not meant to be used directly is called an **abstract class.**

**Exercise**

*Time:* 3 mins.\
*Format:* Individual\
*Post answers:* Yes

- Create a `Triangle` class that inherits from `Shape`. Its constructor (i.e. `__init__`) should take `base` and `height`, and it should have an appropriate `area` method.


```python
class Triangle(Shape):
    def __init__(self, base, height):
        self.base = base
        self.height = height

    def area(self):
        return 1 / 2 * self.base * self.height
```

In [7]:
from shape import Triangle

In [8]:
# import your class and run this cell to test it
assert Triangle(3, 6).area() == 9

$\blacksquare$

## Summary

- A class can **inherit** behavior from a parent class.
- A parent class can be used to implement default behavior that child classes override as appropriate.
- When there is no sensible default, a parent class should raise a `NotImplementedError` that child classes need to override. In an extreme class, an **abstract class** that implements no behavior can still be useful as a template for child classes.